In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

mnist = datasets.MNIST(root='data', download=True, transform=transform)

dataloader = DataLoader(mnist, batch_size=60, shuffle=True)

9913344it [00:01, 7215667.77it/s]                              


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



29696it [00:00, 3705752.63it/s]          


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



1649664it [00:00, 1738904.29it/s]                             


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



5120it [00:00, ?it/s]                   

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



### https://dreamgonfly.github.io/blog/gan-explained/

In [2]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.main = nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=28*28),
            nn.Tanh()
        )

    # (batch_size * 100) 크기의 랜덤 벡터를 받아 이미지를 (batch_size * 1 * 28 * 28) 크기로 출력한다
    def forward(self, inputs):
        return self.main(inputs).view(-1, 1, 28, 28)

In [5]:
# 구분자는 이미지를 입력으로 받아 진짜인지 가짜인지 출력한다.
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )

    #(batch_size * 1 * 28 * 28) 크기의 이미지를 받아 이미지가 진짜일 확률을 0~1사이로 출력한다
    def forward(self, inputs):
        inputs = inputs.view(-1, 28*28)
        return self.main(inputs)
    

In [6]:
# 생성자와 구분자 객체 만들기
G = Generator()
D = Discriminator()

In [ ]:
# 손실함수와 최적화 기법 지정하기
# Binary Cross Entropy Loss
criterion = nn.BCELoss()

# 생성자와 구분자의 매개변수를 최적화하는 Adam Optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 학습을 위한 반복문. 데이터셋을 100번 돌며 학습한다
epochs = 100
for epoch in range(epochs):
    # 한번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
        # 데이터를 파이토치의 변수로 변환
        real_data = Variable(reat_data)
        #
        #
        #
        # 이미지가 진짜일 때 정답 값은 1이고 가짜일때는 0
        # 정답지에 해당하는 변수를 만든다
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
        
        # 진짜이미지를 구분자에 넣는다
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다
        D_loss_real = criterion(D_result_from_real, target_real)
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        # 생성자로 가짜 이미지를 생성
        fake_data = G(z)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개변수의 미분값을 0으로 초기화
        D.zero_grad()
        # 역전파를 통해 매개변수의 loss에 대한 미분값을 계산한다
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개변수를 업데이트한다.
        D_optimizer.step()
        
        ######
        # 생성자 학습.
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다
        z = Variable(torch.randn((batch_size, 100)))
        z = z.cuda()
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        
        # 생성자가 만든 가짜이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다
        G_loss = criterion(D_result_from_fake, target_real)
        # 생성자의 매개변수의 미분값을 0으로 초기화
        G.zero_grad()
        # 역전파를 통해 개개변의 loss에 대한 미분값을 계산
        G.loss.backward()
        # 최적화 기법을 이용 생성자의 매개 변수를 업데이트
        G_optimizer.step()
        